In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

In [2]:
from ckiptagger import WS, POS, NER
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

In [2]:
column_names = ['type','title','text']
df = pd.read_csv('./all_after_mapping.tsv',sep='\t',names=column_names)
# df = df.sample(frac=1)
# df = df.iloc[:int(df.shape[0]*0.5)]

In [4]:
from zhon.hanzi import punctuation
import re
def remove_punctuation(df):
    textarr = df['text'].values
    for i in range(len(textarr)):
        textarr[i] = re.sub("[{}]+".format(punctuation), "",textarr[i])
    df['text'] = textarr
    return df

In [5]:
df = remove_punctuation(df)


In [6]:
def tokenlize(text):
    return ws([text])

In [7]:
alltext = df['text'].apply(tokenlize)
li = []
for i in range(alltext.shape[0]):
    li.append(alltext[i][0])
li = np.array(li
             )
tokenlizeword = np.array(li)
np.save('tokenlizeword0225_nopunct',tokenlizeword)

In [ ]:
tokenlizeword = np.load('tokenlizeword0216.npy',allow_pickle=True)


In [ ]:
np.save('tokenlizeword0118',li)

In [ ]:
tokenlizeword = li

In [3]:
tokenlizeword = np.load('tokenlizeword0221.npy',allow_pickle=True)

In [4]:
wmodel = Word2Vec(tokenlizeword, size=300, window=4, min_count=0)
wmodel.save("word2vec.model")

In [15]:
wmodel.wv.most_similar('^_^')

[('~~~', 0.7460169792175293),
 ('腐鍋', 0.7410570979118347),
 ('⋯⋯⋯', 0.7385553121566772),
 ('回本率', 0.7377778887748718),
 ('讚讚', 0.7315932512283325),
 ('保到', 0.724859356880188),
 ('怡農', 0.722081184387207),
 ('～～', 0.7216964364051819),
 ('聯演', 0.7215677499771118),
 ('休好', 0.7213956117630005)]

In [ ]:
labels = df['type'].values
labels = np.array(labels)
# np.save('labels',labels)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.utils import np_utils
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
from keras_self_attention import SeqSelfAttention




In [ ]:
max_size = 512
x_train = []
for k in range(tokenlizeword.shape[0]):
  # every article have max_size * 300 embedding matrix
    embedding_matrix = np.zeros((max_size,300))
    for i in range(len(tokenlizeword[k])):
        if(i>=max_size):
            break
        embedding_matrix[i] = wmodel[tokenlizeword[k][i]]
    x_train.append(embedding_matrix)

  
  

In [ ]:
x_train = np.array(x_train)
x_train.shape

In [ ]:
x = x_train
y = labels
print(x.shape)
print(y.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=5)
y_train = np_utils.to_categorical(y_train)


In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(512, return_sequences=False), input_shape=(max_size, 300)))
# model.add(TimeDistributed(Dense(7, activation='softmax')))
# model.add(Dense(128,activation='relu'))
# model.add(SeqSelfAttention(attention_activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(7,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=5,verbose=1,validation_split=0.2)

In [ ]:
pred = model.predict(x_test)
pred = np.argmax(pred, axis = 1) # Decode softmax output
accuracy = (pred == y_test).mean()
print('Your test accuracy is %.6f' % (accuracy * 100))

In [ ]:
df_bert = pd.DataFrame({
    'id':np.arange(df.shape[0]),
    'label':df['type'],
    'alpha':['a']*df.shape[0],
    'text': df['text'],
})


In [ ]:
dfdev = df_bert.iloc[0:5000]
dftest = df_bert[5000:10000]
dftrain = df_bert[10000:]
print(dfdev.shape)
print(dftest.shape)
print(dftrain.shape)


In [ ]:
dftrain.tail(10)

In [ ]:
testans = dftest['label']
testans = np.array(testans.values)
np.save('testans',testans)

In [ ]:
# testans = np.load('testans.npy')

In [ ]:
dftrain.to_csv('./bert_data/train.tsv', sep='\t', index=False, header=False)
dfdev.to_csv('./bert_data/dev.tsv', sep='\t', index=False, header=False)
dftest.drop(['label','alpha'],axis=1,inplace=True)
dftest.to_csv('./bert_data/test.tsv', sep='\t', index=False, header=True)

In [ ]:
!sudo rm -r bert_predict/

In [ ]:
!python3 ./bert-master/run_classifier.py \
  --task_name=news \
  --do_train=true \
  --do_eval=true \
  --data_dir=bert_data \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=bert_model_output/model.ckpt-8000 \
  --max_seq_length=512 \
  --train_batch_size=4 \
  --learning_rate=1e-5 \
  --num_train_epochs=2.0 \
  --output_dir=bert_model_output \

In [ ]:
!python3 ./bert-master/run_classifier.py \
  --task_name=news \
  --do_predict=true \
  --data_dir=bert_data \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=bert_model_output/model.ckpt-12773 \
  --max_seq_length=512 \
  --train_batch_size=4 \
  --learning_rate=1e-5 \
  --num_train_epochs=2.0 \
  --output_dir=bert_predict \

In [ ]:
columns = ['0','1','2','3','4','5','6']
preddf = pd.read_csv('./bert_predict/test_results.tsv',sep='\t',names=columns)

In [ ]:
arr = preddf.to_numpy()

In [ ]:
arr = arr.argmax(axis=1)

In [ ]:
arr

In [ ]:
accuracy = (arr == testans).mean()
print('Your test accuracy is %.6f' % (accuracy * 100))

In [8]:
column_names = ['type','title','text']
df_unlabel = pd.read_csv('./udn_for_mct.tsv',sep='\t',names=column_names)

In [10]:
df_for_pretraining = df_unlabel['text']

In [12]:
df_for_pretraining.to_csv('./bert_data/for_pretraining.txt', sep='\t', index=False, header=False)

In [18]:
!python3 ./bert-master/create_pretraining_data.py \
  --input_file=./bert_data/for_pretraining.txt \
  --output_file=./bert_data/for_pretraining_output.tfrecord \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=5


2020-03-09 15:14:43.100010: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-03-09 15:14:43.100400: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-03-09 15:14:43.100408: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
INFO:tensorflow:*** Reading from input files ***
I0309 15:14:44.049029 139801377142592 create_pretraining_data.py:448] *** Reading from input files ***
INFO:tensorflow:  ./bert_data/for_pretraining.txt
I0309 15:14:44.049151 139801377142592 create_pretraining_data.py:450]   ./bert_

INFO:tensorflow:Wrote 42597 total instances
I0309 15:16:16.908919 139801377142592 create_pretraining_data.py:168] Wrote 42597 total instances


In [31]:
!python3 ./bert-master_v2/run_pretraining.py \
  --input_file=./bert_data/for_pretraining_output.tfrecord \
  --output_dir=./bert_data/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=chinese_L-12_H-768_A-12/bert_model.ckpt \
  --train_batch_size=8 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20 \
  --num_warmup_steps=10 \
  --learning_rate=2e-5

2020-03-09 15:45:01.547840: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-03-09 15:45:01.547894: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-03-09 15:45:01.547903: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
INFO:tensorflow:*** Input Files ***
I0309 15:45:02.062458 139683812325184 run_pretraining.py:420] *** Input Files ***
INFO:tensorflow:  ./bert_data/for_pretraining_output.tfrecord
I0309 15:45:02.062572 139683812325184 run_pretraining.py:422]   ./bert_data/for_pretraining_output.tf

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_deterministic`.
W0309 15:45:02.087801 139683812325184 deprecation.py:323] From ./bert-master_v2/run_pretraining.py:368: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_deterministic`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.
W0309 15:45:02.095674 139683812325184 deprecation.py:3

In [24]:
!tf_upgrade_v2 \
  --intree bert-master/ \
  --outtree bert-master_v2/ \
  --reportfile report.txt

2020-03-09 15:26:50.184406: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-03-09 15:26:50.184482: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-03-09 15:26:50.184493: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
ERROR line 30:8: Using member tf.flags in deprecated module tf.flags. tf.flags and tf.app.flags have been removed, please use the argparse or absl modules if you need command line parsing.
INFO line 171:12: Renamed 'tf.trainable_variables' to 'tf.compat.v1.trainable_variables'
INF

ERROR line 28:8: Using member tf.flags in deprecated module tf.flags. tf.flags and tf.app.flags have been removed, please use the argparse or absl modules if you need command line parsing.
WARNING line 62:19: tf.get_variable requires manual check. tf.get_variable returns ResourceVariables by default in 2.0, which have well-defined semantics and are stricter about shapes. You can disable this behavior by passing use_resource=False, or by calling tf.compat.v1.disable_resource_variables().
INFO line 62:19: Renamed 'tf.get_variable' to 'tf.compat.v1.get_variable'
INFO line 64:18: tf.truncated_normal_initializer requires manual check. Initializers no longer have the dtype argument in the constructor or partition_info argument in the __call__ method.
The calls have been converted to compat.v1 for safety (even though they may already have been correct).
INFO line 64:18: Renamed 'tf.truncated_normal_initializer' to 'tf.compat.v1.truncated_normal_initializer'
WARNING line 66:16: tf.get_variable

INFO line 138:25: Renamed 'tf.global_variables_initializer' to 'tf.compat.v1.global_variables_initializer'
INFO line 139:25: Renamed 'tf.local_variables_initializer' to 'tf.compat.v1.local_variables_initializer'
ERROR line 29:8: Using member tf.flags in deprecated module tf.flags. tf.flags and tf.app.flags have been removed, please use the argparse or absl modules if you need command line parsing.
ERROR line 102:0: Using member tf.flags.DEFINE_string in deprecated module tf.flags. tf.flags and tf.app.flags have been removed, please use the argparse or absl modules if you need command line parsing.
ERROR line 108:0: Using member tf.flags.DEFINE_string in deprecated module tf.flags. tf.flags and tf.app.flags have been removed, please use the argparse or absl modules if you need command line parsing.
ERROR line 114:0: Using member tf.flags.DEFINE_string in deprecated module tf.flags. tf.flags and tf.app.flags have been removed, please use the argparse or absl modules if you need command l

ERROR line 27:8: Using member tf.flags in deprecated module tf.flags. tf.flags and tf.app.flags have been removed, please use the argparse or absl modules if you need command line parsing.
INFO line 103:19: Renamed 'tf.python_io.TFRecordWriter' to 'tf.io.TFRecordWriter'
INFO line 151:6: Renamed 'tf.logging.info' to 'tf.compat.v1.logging.info'
INFO line 152:6: Renamed 'tf.logging.info' to 'tf.compat.v1.logging.info'
INFO line 162:8: Renamed 'tf.logging.info' to 'tf.compat.v1.logging.info'
INFO line 168:2: Renamed 'tf.logging.info' to 'tf.compat.v1.logging.info'
INFO line 194:9: Renamed 'tf.gfile.GFile' to 'tf.io.gfile.GFile'
INFO line 439:2: Renamed 'tf.logging.set_verbosity' to 'tf.compat.v1.logging.set_verbosity'
INFO line 439:27: Renamed 'tf.logging.INFO' to 'tf.compat.v1.logging.INFO'
INFO line 446:23: Renamed 'tf.gfile.Glob' to 'tf.io.gfile.glob'
INFO line 448:2: Renamed 'tf.logging.info' to 'tf.compat.v1.logging.info'
INFO line 450:4: Renamed 'tf.logging.info' to 'tf.compat.v1.log

INFO line 93:9: Renamed 'tf.gfile.GFile' to 'tf.io.gfile.GFile'
INFO line 171:9: Renamed 'tf.variable_scope' to 'tf.compat.v1.variable_scope'
INFO line 172:11: Renamed 'tf.variable_scope' to 'tf.compat.v1.variable_scope'
INFO line 196:11: Renamed 'tf.variable_scope' to 'tf.compat.v1.variable_scope'
INFO line 224:11: Renamed 'tf.variable_scope' to 'tf.compat.v1.variable_scope'
INFO line 228:29: Renamed 'tf.layers.dense' to 'tf.compat.v1.layers.dense'
INFO line 358:11: Changing keep_prob arg of tf.nn.dropout to rate, and recomputing value.

WARNING line 364:9: tf.contrib.layers.layer_norm requires manual check. (Manual edit required) `tf.contrib.layers.layer_norm` has been deprecated, and its implementation has been integrated with `tf.keras.layers.LayerNormalization` in TensorFlow 2.0. Note that, the default value of `epsilon` is changed to `1e-3` in the new API from `1e-12`, and this may introduce numerical differences. Please check the new API and use that instead.
ERROR line 364:9: U